In [5]:
import requests
import pandas as pd
import json
import matplotlib.pyplot as plt
from datetime import datetime

# Analyzing Aggravated Burglaries in Davidson County

### Part 1 - Data Gathering using APIs

1. A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents () that were reported during the nine month period from January 1, 2022 through September 30, 2022. (**Hint:** Check out the [API Docs](https://dev.socrata.com/foundry/data.nashville.gov/2u6v-ujjs) to see how to narrow down the response to just the desired results).

**Aggrivated Burglary** - TIBRS Data Collection Manual states that TN uses NIBRS for coding offenses under TN Code Annotated 39-14-403.  The following NIBRS codes are used for aggrivated burglary in Tennessee
- Burglary - 220
- Assualt - 13A
- Robery - 120
- Weapon Law Violation - 520

In [7]:
#Setup API Access
with open('app_token.json') as fi:
    credentials = json.load(fi)
api_key = credentials['token']

#########
#delete me
###########


API Fields
- `incident_occurred` - Date and time the incident occurred
    - $where=date between `start` and `stop`
    - ISO8601 Times (GMT-5)
        - **1.1.22** `2022-01-01T00:00:00-0500`
        - **9.30.22** `2022-09-30T23:59:59-0500`
- date = `'date between '2015-01-10T12:00:00' and '2015-01-10T14:00:00'`
    

In [56]:
#use 220 for now.  More filters are better.
Agg_Bur = ['220', '13A', '120', '520']
dates = ("'2022-01-01T00:00:00'", "'2022-09-30T23:59:59'")
#HOWTOFINDINCIDENTOCCURRED = f'incident_occurred between''2015-01-10T12:00:00'' and ''2015-01-10T14:00:00''
#originaldates = f'$where=incident_occurred between '+start+' and '+stop
print(dates[0])


'2022-01-01T00:00:00'


In [58]:
# By default, it's only going to give you 100 Rows back
# https://dev.socrata.com/docs/paging.html#2.1 - up the limit of responses
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.json'
params = {
    '$where': f'incident_occurred between '+dates[0]+' and '+dates[1],
    #offense_nibrs#
    '$$app_token': api_key
}
response = requests.get(endpoint)
###################Original 
#params = {'incident_occurred': dates}
#response = requests.get(endpoint, params = params)
#'offense_nibrs' : Agg_Bur,
#'$$app_token': app_token

In [59]:
response = requests.get(endpoint, params = params)

In [60]:
response

<Response [200]>

In [61]:
res = response.json()
res

[{'primary_key': '20220002526_11',
  'incident_number': '20220002526',
  'report_type': 'D',
  'report_type_description': 'DISPATCHED',
  'incident_status_code': 'O',
  'incident_status_description': 'OPEN',
  'investigation_status': 'Open',
  'incident_occurred': '2022-01-01T00:00:00.000',
  'incident_reported': '2022-01-03T00:50:00.000',
  'incident_location': '4TH AVE N',
  'latitude': '36.17',
  'longitude': '-86.79',
  'rpa': '4235',
  'zone': '413',
  'location_code': '90',
  'location_description': 'APARTMENT',
  'offense_number': '1',
  'offense_nibrs': '23D',
  'offense_description': 'LARC - FROM BLDG',
  'weapon_primary': '17',
  'weapon_description': 'NONE',
  'victim_number': '1',
  'domestic_related': False,
  'victim_type': 'I',
  'victim_description': 'INDIVIDUAL (18 AND OVER)',
  'victim_gender': 'M',
  'victim_race': 'W',
  'victim_ethnicity': 'Non-Hispanic',
  'victim_county_resident': 'RESIDENT',
  'mapped_location': {'type': 'Point', 'coordinates': [-86.79, 36.17]}}

In [62]:
df = pd.DataFrame(res)
df

,primary_key,incident_number,report_type,report_type_description,incident_status_code,incident_status_description,investigation_status,incident_occurred,incident_reported,incident_location,...,victim_number,domestic_related,victim_type,victim_description,victim_gender,victim_race,victim_ethnicity,victim_county_resident,mapped_location,zip_code
0,20220002526_11,20220002526,D,DISPATCHED,O,OPEN,Open,2022-01-01T00:00:00.000,2022-01-03T00:50:00.000,4TH AVE N,...,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.79, 36.17]}",NaN
1,20220047605_11,20220047605,D,DISPATCHED,U,UNFOUNDED,Closed,2022-01-01T00:00:00.000,2022-02-01T10:33:00.000,7716 7716,...,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Hispanic,RESIDENT,NaN,37080
2,20220046961_11,20220046961,D,DISPATCHED,U,UNFOUNDED,Closed,2022-01-01T00:00:00.000,2022-02-01T02:52:00.000,2209 2209,...,1,False,I,INDIVIDUAL (18 AND OVER),M,B,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.82, 36.1...",37215
3,20220056911_11,20220056911,S,SUSPECT,O,OPEN,Open,2022-01-01T00:00:00.000,2022-02-07T07:52:00.000,HOBSON PIKE,...,1,False,S,SOCIETY,NaN,NaN,NaN,NaN,"{'type': 'Point', 'coordinates': [-86.6, 36.05]}",NaN
4,20220000685_11,20220000685,D,DISPATCHED,O,OPEN,Open,2022-01-01T00:00:00.000,2022-01-01T13:43:00.000,SHELBY AVE,...,1,False,I,INDIVIDUAL (18 AND OVER),M,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.74, 36.17]}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,20220003810_11,20220003810,D,DISPATCHED,A,CLEARED BY ARREST,Closed,2022-01-03T20:00:00.000,2022-01-03T22:04:00.000,3036 3036,...,1,True,I,INDIVIDUAL (18 AND OVER),F,B,Non-Hispanic,NON RESIDENT,"{'type': 'Point', 'coordinates': [-86.823, 36....",37218
996,20220003772_11,20220003772,D,DISPATCHED,U,UNFOUNDED,Closed,2022-01-03T20:01:00.000,2022-01-03T20:35:00.000,3458 3458,...,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Unknown,RESIDENT,"{'type': 'Point', 'coordinates': [-86.757, 36....",37207
997,20220003902_21,20220003902,D,DISPATCHED,O,OPEN,Open,2022-01-03T20:13:00.000,2022-01-04T00:13:00.000,LEBANON PIKE,...,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.63, 36.19]}",NaN
998,20220003902_11,20220003902,D,DISPATCHED,O,OPEN,Open,2022-01-03T20:13:00.000,2022-01-04T00:13:00.000,LEBANON PIKE,...,1,False,I,INDIVIDUAL (18 AND OVER),F,W,Non-Hispanic,RESIDENT,"{'type': 'Point', 'coordinates': [-86.63, 36.19]}",NaN


2. Using the [2020 American Community Survey API](https://www.census.gov/data/developers/data-sets/acs-5year.html), obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37. 

### Part 2 - Spatial Joining and Data Merging

3. Download the 2020 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. 

4. Aggregate the data by census tract. **Warning:** each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents. Which census tract had the highest number of burglaries? Which census tract had the highest number of burglaries per 1000 residents? **Note:** Make sure that you keep all census tracts, not just those that have had a burglary.

5. Merge in the census data that you gathered in question 2. Remove any rows that have zero population or negative median income values.

### Part 3 - Statistical Modeling

6. Finally, we'll build some statistical models to see how well we can explain the number of aggravated burglaries using the median income of each census tract. Start with some EDA to look at the relationship between median income and number of aggravated burglaries.

7. Fit a Poisson regression model with target variable the rate of burglaries per census tract and with predictor the median income. Offset using the log of the population so that we are looking at the rate of burglaries per population instead of the number of burglaries. How can you interpret the meaning of the output?

8. **Bonus:** Try out a negative binomial model. To get started with a negative binomial model, you can check out [this tutorial](https://timeseriesreasoning.com/contents/negative-binomial-regression-model/). How does this model compare to the Poisson model?

Additional Resources for Generalized Linear Models:
* DataCamp - [Generalized Linear Models in Python](https://learn.datacamp.com/courses/generalized-linear-models-in-python)
* [Beyond Multiple Linear Regression, Chapter 4](https://bookdown.org/roback/bookdown-BeyondMLR/ch-poissonreg.html) Warning - the code in this book is all R, but the conceptual explanations are very clear.
* [This set of notes](https://apwheele.github.io/MathPosts/PoissonReg.html#negative-binomial-when-the-poisson-does-not-fit), which talks about the problem of overdispersion.